## Cadences Network Visualization with Louvain Communities



In [3]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [2]:
# Example using batch to count the cadence types from multiple pieces:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei'
                    ,'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
                    'https://crimproject.org/mei/CRIM_Mass_0019_5.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["shifted_all_feat"] = combined_df.groupby("Title")['all_features'].shift(-1)

Successfully imported https://crimproject.org/mei/CRIM_Model_0019.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0019_5.mei


In [4]:
combined_df = pd.read_csv ("saved_csv/all_cads.csv")

In [6]:
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
combined_df.copy
combined_df["shifted_all_feat"] = combined_df.groupby("Title")['all_features'].shift(-1)

### Network Visualization with Louvain Communities

In [7]:
combined_df = combined_df.dropna(subset=['all_features', 'shifted_all_feat'])
cadence_pairs = list(zip(combined_df.all_features, combined_df.shifted_all_feat))

In [8]:
combined_df['all_features'].nunique()

166

In [9]:
cadence_pairs

[('Authentic_D_False_1.0', 'Authentic_D_False_1.0'),
 ('Authentic_D_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_D_False_2.0'),
 ('Authentic_D_False_2.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_D_False_1.0'),
 ('Authentic_D_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_G_False_3.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_G_False_3.0'),
 ('Clausula Vera_D_False_1.0', 'Authentic_G_False_1.0'),
 ('Clausula Vera_D_False_3.0', 'Authentic_G_False_3.0'),
 ('Authentic_G_False_3.0', 'Authentic_C_False_1.0'),
 ('Authentic_C_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Authentic_D_False_1.0'),
 ('Authentic_D_False_1.0', 'Authentic_G_False_1.0'),
 ('Authentic_G_False_1.0', 'Clausula Vera_D_False_1.0'),
 ('Clausula Vera_D_False_1.0', 'Clausula Vera_D_False_1.0'),
 ('Clausula Vera_D_Fal

In [11]:
combined_df['all_features'].nunique()

35

In [11]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black", directed=True)

In [12]:
# G = nx.Graph()
G = nx.DiGraph()
# G.add_edges_from(unique_pairs)
G.add_edges_from(cadence_pairs)
# G = add_communities(G)
pyvis_graph.from_nx(G)

In [14]:
# pyvis_graph.show('all_cads.html')


In [41]:
mass_sets = [('CRIM_Mass_0019',
('CRIM_Model_0019.mei',
 'CRIM_Mass_0019_1.mei',
 'CRIM_Mass_0019_2.mei',
 'CRIM_Mass_0019_3.mei',
 'CRIM_Mass_0019_4.mei',
 'CRIM_Mass_0019_5.mei'))]



In [42]:
mass_sets[0][0]

'CRIM_Mass_0019'

In [48]:
for mass in mass_sets:
    corpus_set = []
    csv_name = 'saved_csv/' + mass_sets[0][0] + '_' + 'cadences'
    file_names = mass[1]
    for file in file_names:
        full_file_name = 'https://crimproject.org/mei/' + file
        corpus_set.append(full_file_name)
    
    corpus = CorpusBase(corpus_set)
    list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
    combined_df = pd.concat(list_of_dfs, ignore_index=True)
    combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str) + "_" + combined_df['Beat'].apply(str)
    combined_df['some_features'] = combined_df['CadType'] + "_" + combined_df['Tone']
# return(combined_df)
        

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [49]:
combined_df.to_csv(csv_name)